In [128]:
import numpy as np
import itertools as it

def polyadd2(x, y):
    return np.polyadd(x, y) % 2

def polysub2(x, y):
    return np.polysub(x, y) % 2
    
def polymul2(x, y):
    return np.polymul(x, y) % 2
    
def polydiv2(x, y):
    return np.polydiv(x, y)[0] % 2
    
def polymod2(x, y):
    return np.polydiv(x, y)[1] % 2

def drop_trailing_zeros(l):
    for i in range(len(l)):
        if l[i] == 1:
            return l[i:]
    return [0]    

def find_divisor(poly):
    n = len(poly)
    if n in [0, 1]:
        return []
    divisors = list(map(lambda x: drop_trailing_zeros(list(x)), it.product([0, 1], repeat=n)))
    res = []
    for divis in divisors:
#         print("Poly: {}".format(divis))
#         print("Mod: {}".format(polymod2(poly, divis)))
        if all(map(lambda x: x == 0, polymod2(poly, divis))):
            res.append(divis)
    return res        
        

def find_prime_div(poly):
    divis = find_divisor(poly)
    return list(filter(lambda x: len(find_divisor(x)) == 2, divis))
  
# check
# 1 + x^7
# = (1 + x)(1 + x + x^3)(1 + x^2 + x^3)
# = [1, 1];[1, 1, 0, 1];[1, 0, 1, 1]
print(find_prime_div([1, 0, 0, 0, 0, 0, 0, 1]))

[[1, 1], [1, 0, 1, 1], [1, 1, 0, 1]]


In [129]:
import functools as fp

n = 9
xn_1 = [1 if i in [0, n] else 0 for i in range(n + 1)]
divs = find_prime_div(xn_1)
print("Prime divisors: {}".format(divs))
product = fp.reduce(polymul2, divs[1:], divs[0])
print("Product of divisors: {}".format(product))
print("Product len: {}".format(len(product)))

Prime divisors: [[1, 1], [1, 1, 1], [1, 0, 0, 1, 0, 0, 1]]
Product of divisors: [1 0 0 0 0 0 0 0 0 1]
Product len: 10


In [216]:
def encode(n, x, poly):
    k = len(x)
    x_nk = [1 if i == 0 else 0 for i in range(n - k + 1)]
    xmul = polymul2(x_nk, x)
    t = polymod2(drop_trailing_zeros(xmul), poly)
    res = list(map(int, polyadd2(xmul, t)))
    return [0 for i in range(n - len(res))] + res

# 11011000010 ->
# 110110000101011
n = 15
x14_1 = [1 if i in [0, n - 1] else 0 for i in range(n)]
x = [1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0]
poly = [1, 0, 0, 1, 1]
print("Encoded: {}".format(encode(n, x, poly)))
ans = [1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1]
print("Real:    {}".format(ans))

Encoded: [1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1]
Real:    [1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1]


In [233]:
def find_min_d(codes):
    min_d = 1e9;
    min_code = None
    for code in codes:
        w = sum(code)
        if w != 0 and w < min_d:
            min_d = w
            min_code = code
    
    return min_d, min_code

n = 5
k = 3
x_n1 = [1 if i in [0, n] else 0 for i in range(n + 1)]
divis = find_prime_div(x_n1)
for div in divis:
    k = n - len(div) + 1
    inputs = list(map(list, it.product([0, 1], repeat=k)))
    codes = []
    for inp in inputs:
        c = encode(n, inp, div)
        codes.append(c)
    codes = np.array(codes)    
    print("g(x):")
    print(np.poly1d(div))
    print("m = {}".format(len(codes)))
    print(codes)
    min_d, min_codes = find_min_d(codes)
    print("Mid d: {}".format(min_d))
    print("{} ~ {}".format(np.zeros(n, dtype=int), min_codes))
    print()

g(x):
 
1 x + 1
m = 16
[[0 0 0 0 0]
 [0 0 0 1 1]
 [0 0 1 0 1]
 [0 0 1 1 0]
 [0 1 0 0 1]
 [0 1 0 1 0]
 [0 1 1 0 0]
 [0 1 1 1 1]
 [1 0 0 0 1]
 [1 0 0 1 0]
 [1 0 1 0 0]
 [1 0 1 1 1]
 [1 1 0 0 0]
 [1 1 0 1 1]
 [1 1 1 0 1]
 [1 1 1 1 0]]
Mid d: 2
[0 0 0 0 0] ~ [0 0 0 1 1]

g(x):
   4     3     2
1 x + 1 x + 1 x + 1 x + 1
m = 2
[[0 0 0 0 0]
 [1 1 1 1 1]]
Mid d: 5
[0 0 0 0 0] ~ [1 1 1 1 1]



In [247]:
def get_field(n, poly):
    cur = np.zeros(n, dtype=int)
    cur[-1] = 1
    st = cur.copy()
    field = [st]
    while True:
        cur = list(map(int, polymod2(polymul2([1, 0], cur), poly)))
        cur = [0 for i in range(n - len(cur))] + cur
        if (cur == st).all():
            break
        field.append(cur)
    return np.array(field)

# 001
# 010
# 100
# 011
# 110
# 111
# 101
print(get_field(3, [1, 0, 1, 1]))
print()

# 0001
# 0010
# 0100
# 1000
# 0011
# 0110
# 1100
# 1011
# 0101
# 1010
# 0111
# 1110
# 1111
# 1101
# 1001
print(get_field(4, [1, 0, 0, 1, 1]))

[[0 0 1]
 [0 1 0]
 [1 0 0]
 [0 1 1]
 [1 1 0]
 [1 1 1]
 [1 0 1]]

[[0 0 0 1]
 [0 0 1 0]
 [0 1 0 0]
 [1 0 0 0]
 [0 0 1 1]
 [0 1 1 0]
 [1 1 0 0]
 [1 0 1 1]
 [0 1 0 1]
 [1 0 1 0]
 [0 1 1 1]
 [1 1 1 0]
 [1 1 1 1]
 [1 1 0 1]
 [1 0 0 1]]


In [255]:
field = get_field(4, [1, 0, 0, 1, 1])
g_x = [1]
for i in range(4):
print(np.poly1d(g_x))  

 
1
 
1 x + 1



 
1 x + 1
 
0



 
0
   2
1 x + 1 x



 
0
   3
1 x + 1 x



 
0


In [359]:
def polyadd3(x, y):
    return np.polyadd(x, y) % 3

def polymul3(x, y):
    return np.polymul(x, y) % 3
    
def polymod3(x, y):
    return np.polydiv(x, y)[1] % 3

def get_field3(n, poly):
    cur = np.zeros(n, dtype=int)
    cur[-1] = 1
    st = cur.copy()
    field = [st]
    while True:
        cur = list(map(int, polymod3(polymul3([1, 0], cur), poly)))
        cur = [0 for i in range(n - len(cur))] + cur
        if (cur == st).all():
            break
        field.append(cur.copy())
    return np.array(field)


def print_field(field):
    for i in range(len(field)):
        cur = field[i]
        print("{} {} | ".format(cur, i).rjust(13), end='')
#         print("{} | ".format(i).rjust(5), end='')
        if cur[0] != 0:
            p = "" if cur[0] == 1 else cur[0]
            print("{}a^2 ".format(p), end='')
        if cur[1] != 0:
            if cur[0] != 0:
                print("+ ", end='')
            p = "" if cur[1] == 1 else cur[1]    
            print("{}a ".format(p), end='')    
        if cur[2] != 0 or (cur[0] == 0 and cur[1] == 0):
            if cur[0] != 0 or cur[1] != 0:
                print("+ ", end='')
            print("{}".format(cur[2]), end='')  
        print()    

g = [1, 2, 1, 1]
field = get_field3(3, g)
print_field(field)

 [0 0 1] 0 | 1
 [0 1 0] 1 | a 
 [1 0 0] 2 | a^2 
 [1 2 2] 3 | a^2 + 2a + 2
 [0 1 2] 4 | a + 2
 [1 2 0] 5 | a^2 + 2a 
 [0 2 2] 6 | 2a + 2
 [2 2 0] 7 | 2a^2 + 2a 
 [1 1 1] 8 | a^2 + a + 1
 [2 0 2] 9 | 2a^2 + 2
[2 0 1] 10 | 2a^2 + 1
[2 2 1] 11 | 2a^2 + 2a + 1
[1 2 1] 12 | a^2 + 2a + 1
[0 0 2] 13 | 2
[0 2 0] 14 | 2a 
[2 0 0] 15 | 2a^2 
[2 1 1] 16 | 2a^2 + a + 1
[0 2 1] 17 | 2a + 1
[2 1 0] 18 | 2a^2 + a 
[0 1 1] 19 | a + 1
[1 1 0] 20 | a^2 + a 
[2 2 2] 21 | 2a^2 + 2a + 2
[1 0 1] 22 | a^2 + 1
[1 0 2] 23 | a^2 + 2
[1 1 2] 24 | a^2 + a + 2
[2 1 2] 25 | 2a^2 + a + 2


In [376]:
m = 26
def field_add(x, y):
    x = x % m
    y = y % m
    res = polymod3(polyadd3(field[x], field[y]), g)
    res = np.array([0 for i in range(3 - len(res))] + list(res))
    for i in range(len(field)):
        if (field[i] == res).all():
            return i
    return -1   

def field_mul(x, y):
    x = x % m
    y = y % m
    res = polymod3(polymul3(field[x], field[y]), g)
    res = np.array([0 for i in range(3 - len(res))] + list(res))
    for i in range(len(field)):
        if (field[i] == res).all():
            return i
    return -1 

def field_div(x, y):
    x = x % m
    for i in range(len(field)):
        if x == field_mul(i, y):
            return i
    return -1    

# (a^47 + a^69) * a^52 / (a^113 + a^259)
print("(a^{} + a^{}) * a^{} / (a^{} + a^{})".format(47 % m, 69 % m, 52 % m, 113 % m, 259 % m))
print("a^{} / a^{}".format(field_add(21, 17), field_add(9, 25)))
print("a^{}".format(field_div(18, 8)))

(a^21 + a^17) * a^0 / (a^9 + a^25)
a^18 / a^8
a^10


In [387]:
def cyclotomic(n):
    cnt = 0
    was = set()
    res = []
    while cnt < n:
        cur_s = set()
        for i in range(1, n + 1):
            if not(i in was):
                cur = i
                break
        while not (cur in cur_s):
            cnt += 1
            was.add(cur)
            cur_s.add(cur)
            cur = cur * 3 % n
        res.append(list(cur_s))
    return np.array(res)


cycl = cyclotomic(26)
for c in cycl:
    print(c)
    
print()
merge = sorted(cycl[5] + cycl[8])
print(merge)

print("M_(a^8) = (x + a^8) * (x + a^24) * (x + a^20)")
print("a^8 * a^24 = a^{}; a^8 * a^20 = a^{}; a^24 * a^20 = a^{};".format(field_mul(8, 24),
                                                                         field_mul(8, 20),
                                                                         field_mul(24, 20)))
print("a^8 * a^24 * a^20 = a^{}".format(field_mul(8, field_mul(24, 20))))
print("M_(a^8) = x^3 + (a^24 + a^20 + a^8) * x^2 + (a^6 + a^2 + a^18) * x + a^0")
print("M_(a^8) = x^3 + a^{} * x^2 + a^{} * x + a^0".format(field_add(24, field_add(20, 8)), 
                                                            field_add(6, field_add(2, 18))))
print("M_(a^8) = x^3 + a^13 * x + 1")
print()
print("M_(a^17) = (x + a^17) * (x + a^25) * (x + a^23)")
print("M_(a^17) = x^3 + (a^25 + a^23 + a^17) * x^2 + (a^48 + a^42 + a^40) * x + a^65")
print("M_(a^17) = x^3 + a^{} * x^2 + a^{} * x + a^{}".format(field_add(25, field_add(23, 7)), 
                                                             field_add(48, field_add(42, 40)),
                                                             65 % m))

print()
print(field_add(26, 13))

# x^6 + a^10 * x^5 + 2 a^13 x^4 + a^19 * x^3 + a^9 x^2 + a^13

[1, 3, 9]
[2, 18, 6]
[10, 4, 12]
[19, 5, 15]
[11, 21, 7]
[8, 24, 20]
[13]
[16, 14, 22]
[17, 25, 23]
[0, 26]

[8, 17, 20, 23, 24, 25]
M_(a^8) = (x + a^8) * (x + a^24) * (x + a^20)
a^8 * a^24 = a^6; a^8 * a^20 = a^2; a^24 * a^20 = a^18;
a^8 * a^24 * a^20 = a^0
M_(a^8) = x^3 + (a^24 + a^20 + a^8) * x^2 + (a^6 + a^2 + a^18) * x + a^0
M_(a^8) = x^3 + a^-1 * x^2 + a^13 * x + a^0
M_(a^8) = x^3 + a^13 * x + 1

M_(a^17) = (x + a^17) * (x + a^25) * (x + a^23)
M_(a^17) = x^3 + (a^25 + a^23 + a^17) * x^2 + (a^48 + a^42 + a^40) * x + a^65
M_(a^17) = x^3 + a^10 * x^2 + a^13 * x + a^13

-1


/home/Valeriy.Teplyakov/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [ ]:
def encode(n, x, poly):
    k = len(x)
    x_nk = [1 if i == 0 else 0 for i in range(n - k + 1)]
    xmul = polymul3(x_nk, x)
    t = polymod3(drop_trailing_zeros(xmul), poly)
    res = list(map(int, polyadd3(xmul, t)))
    return [0 for i in range(n - len(res))] + res

# 11011000010 ->
# 110110000101011
n = 15
x14_1 = [1 if i in [0, n - 1] else 0 for i in range(n)]
x = [1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0]
poly = [1, 0, 0, 1, 1]
print("Encoded: {}".format(encode(n, x, poly)))
ans = [1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1]
print("Real:    {}".format(ans))

In [419]:
# Найдите все НЕпримитивные элементы в поле GF(3^2), заданном как кольцо вычетов по модулю p(x) = x^2+1
g = [1, 0, 1]
field = get_field3(2, g)
a = []
for el in field:
    cur_set = set()
    res = []
    cur = el
    cur_s = str(el)
    while not (cur_s in cur_set):
        cur_set.add(cur_s)
        res.append(list(map(int, cur)))
        cur = polymod3(polymul3(cur, el), g)
        cur = [0 for i in range(3 - len(cur))] + cur
        cur_s = str(cur)
    a.append(res)
             
for e in a:
    print(e)

[[0, 1], [1, 1]]
[[1, 0], [2, 2], [2, 1], [1, 1], [1, 2]]
[[0, 2], [1, 1], [2, 2]]
[[2, 0], [2, 2], [1, 2], [1, 1], [2, 1]]


In [401]:
import sympy as sy

def apow(n):
    deg = 3
    q = list(map(int, np.mod(np.polydiv([1] + [0] * n, poly)[1], 3)))
    if len(q) < deg:
        q = [0] * (deg - len(q)) + q
    a = sy.Symbol('a')
    s = 0
    w = 2
    for c in q:
        s += int(c)*a**w
        w -= 1
    return s

a = sy.Symbol('a')
spoly = a^3 + 2*a^2 + a + 1

sy.rem(sy.expand(sy.div((apow(57)+apow(68))*apow(52), apow(113)*apow(259))[0], spoly))

TypeError: rem() missing 1 required positional argument: 'g'